In [ ]:
from bs4 import BeautifulSoup
import base64
import re
import random
import pytesseract
from PIL import Image, ImageFilter
import cv2
from collections import Counter
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from transformers import pipeline


In [75]:
qa_pipeline = pipeline("question-answering")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [76]:
def preprocess_captcha(image_path):
    # Read the image
    image = cv2.imread(image_path)
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply thresholding (binarization)
    _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    
    # Optionally, apply noise removal (median filter)
    denoised = cv2.medianBlur(binary, 3)
    
    # Save the preprocessed image for debugging
    preprocessed_path = "preprocessed_captcha.png"
    cv2.imwrite(preprocessed_path, denoised)
    
    return preprocessed_path

def read_captcha(captcha_path):
    # Preprocess the captcha image
    preprocessed_path = preprocess_captcha(captcha_path)
    
    # Read the preprocessed image with pytesseract
    captcha_text = pytesseract.image_to_string(Image.open(preprocessed_path), config="--psm 7")
    
    return captcha_text.strip()

def run_multiple_captcha_attempts(captcha_patch,attempts=10):
    results = []
    
    for i in range(attempts):
        # print(f"Attempt {i + 1}...")
        # Download captcha
        # if download_captcha(captcha_url):  # This correctly downloads the captcha image
            # Read and process the captcha
            captcha_text = read_captcha(captcha_patch)
            results.append(captcha_text)
            # print(f"Extracted CAPTCHA (Attempt {i + 1}): {captcha_text}")
    print(results)
    # Determine the most repeated result
    most_common = Counter(results).most_common(1)
    if most_common:
        print(f"\nMost repeated CAPTCHA result: {most_common[0][0].replace(" ", "")}")
    
    return most_common[0][0].replace(" ", "")

def gen_captcha_text():
    try:
        time.sleep(2)
        label = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CLASS_NAME, "wt-captcha--text-challenge"))
        )
        label_html = label.get_attribute("outerHTML")
        # print("Nội dung của label:", label_html)
    except Exception as e:
        print("Không tìm thấy thẻ label:", e)
    soup = BeautifulSoup(label_html, 'html.parser')
    base64_string = soup.find('img' , alt="First number")['src']
    base64_string = base64_string.replace('data:image/png;base64,','')
    image_data = base64.b64decode(base64_string)
    with open("img1.png", "wb") as file:
        file.write(image_data)
    base64_string2 = soup.find('img' , alt="Second number")['src']
    base64_string2 = base64_string2.replace('data:image/png;base64,','')
    image_data2 = base64.b64decode(base64_string2)
    with open("img2.png", "wb") as file:
        file.write(image_data2)
    img1_txt  = run_multiple_captcha_attempts("img1.png")
    img2_txt  = run_multiple_captcha_attempts("img2.png")
    # Thay thế thẻ <img> có alt="First number"
    img1_tag = soup.find("img", alt="First number")
    if img1_tag:
        img1_tag.replace_with(img1_txt)

    # Thay thế thẻ <img> có alt="second number"
    img2_tag = soup.find("img", alt="Second number")
    if img2_tag:
        img2_tag.replace_with(img2_txt)
    captcha_text = soup.text.replace('\n','').strip()

    print(captcha_text)
    return captcha_text
def extract_number_ai(captcha_text):
    result = qa_pipeline({
        "question": "What number should I move the slider to?",
        "context": captcha_text
    })
    
    # Lấy tất cả các số trong kết quả
    numbers = list(map(int, re.findall(r'\d+', result["answer"])))
    
    if not numbers:
        raise ValueError("Không tìm thấy số nào trong kết quả AI.")
    
    # Nếu có nhiều số, lấy trung bình
    return round(sum(numbers) / len(numbers))

def solve_captcha(driver):
    """Hàm giải captcha và kéo slider"""
    print("🛠️ Giải Captcha...")

    # Lấy text từ captcha bằng AI
    captcha_text = gen_captcha_text()
    try:
        answer_number = extract_number_ai(captcha_text)
        
    except:
        answer_number = random.randint(0, 50)
    print(f"🔢 Số cần kéo đến: {answer_number}")
    # Tìm phần tử slider
    slider = driver.find_element(By.CLASS_NAME, "wt-captcha__slider-slider")
    slider_width = slider.size["width"]
    pixel_for_move = slider_width / 50 * answer_number - 165  # Điều chỉnh vị trí kéo

    # Kéo slider
    action = ActionChains(driver)
    action.click_and_hold(slider).move_by_offset(pixel_for_move, 0).release().perform()
    print("➡️ Đã kéo slider.")

    # Nhấn nút Submit
    submit_button = driver.find_element(By.CSS_SELECTOR, 'button.ecl-button--call[type="submit"]')
    driver.execute_script("arguments[0].click();", submit_button)
    print("📩 Đã nhấn Submit.")

In [77]:
# Mở trình duyệt
print("🔓 Mở trình duyệt để nhập captcha...")
driver = webdriver.Chrome()
driver.get("https://trade.ec.europa.eu/access-to-markets/en/captcha?redirectTo=%2Fen%2Fsearch%3Fproduct%3D0101%26origin%3DVN%26destination%3DAT")
print("⌨️ Vui lòng nhập captcha, sau đó đóng trình duyệt nếu không cần chạy lại.")

max_attempts = 10  # Số lần thử tối đa
attempt = 0

while attempt < max_attempts:
    try:
        solve_captcha(driver)  # Giải captcha

        # Kiểm tra lỗi xuất hiện hay không
        error_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CLASS_NAME, "info-error"))
        )
        print(f"⚠️ Thử lần {attempt + 1}: Captcha không đúng, thử lại...")
        attempt += 1
        time.sleep(3)  # Đợi trước khi thử lại

    except TimeoutException:
        # Nếu không có lỗi xuất hiện, captcha đã được giải thành công
        print("✅ Captcha đã được giải thành công!")
        driver.quit()
        break  # Thoát vòng lặp

# Nếu sau 10 lần vẫn không được, chờ 10 phút để người dùng tự giải
if attempt == max_attempts:
    print("⏳ Thử 10 lần không thành công. Chờ 10 phút để người dùng tự giải...")
    # Đo thời gian bắt đầu
    start_captcha_time = time.time()
    wait_time = 60  # 30 phút 

    # Chờ người dùng tự đóng trình duyệt, nếu quá 10 phút thì tự động đóng
    while True:
        try:
            if time.time() - start_captcha_time > wait_time:
                print(f"⏳ Quá {wait_time/60} phút, tự động đóng trình duyệt.")
                driver.quit()
                break  # Thoát khỏi vòng lặp

            # Kiểm tra trình duyệt còn mở không
            driver.title  
            
            # Kiểm tra trang có bị treo không
            if driver.execute_script("return document.readyState") != "complete":
                print("⚠️ Trang web có thể bị treo, chờ thêm...")
            
        except WebDriverException:
            print("✅ Trình duyệt đã được đóng.")
            break  # Nếu trình duyệt bị đóng, thoát vòng lặp
        
        time.sleep(5)  # Tránh vòng lặp chạy quá nhanh, gây tải CPU

🔓 Mở trình duyệt để nhập captcha...
⌨️ Vui lòng nhập captcha, sau đó đóng trình duyệt nếu không cần chạy lại.
🛠️ Giải Captcha...
['x7', 'x7', 'x7', 'x7', 'x7', 'x7', 'x7', 'x7', 'x7', 'x7']

Most repeated CAPTCHA result: x7
['90', '90', '90', '90', '90', '90', '90', '90', '90', '90']

Most repeated CAPTCHA result: 90
Move the slider to x7 or 90.
🔢 Số cần kéo đến: 48


c:\Users\nguye\anaconda3\Lib\site-packages\transformers\pipelines\question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


➡️ Đã kéo slider.
📩 Đã nhấn Submit.
⚠️ Thử lần 1: Captcha không đúng, thử lại...
🛠️ Giải Captcha...
['eight', 'eight', 'eight', 'eight', 'eight', 'eight', 'eight', 'eight', 'eight', 'eight']

Most repeated CAPTCHA result: eight
['3', '3', '3', '3', '3', '3', '3', '3', '3', '3']

Most repeated CAPTCHA result: 3
Use the slider to select the lower number: eight or 3.
🔢 Số cần kéo đến: 3
➡️ Đã kéo slider.
📩 Đã nhấn Submit.
⚠️ Thử lần 2: Captcha không đúng, thử lại...
🛠️ Giải Captcha...
['- twenty-four:', '- twenty-four:', '- twenty-four:', '- twenty-four:', '- twenty-four:', '- twenty-four:', '- twenty-four:', '- twenty-four:', '- twenty-four:', '- twenty-four:']

Most repeated CAPTCHA result: -twenty-four:
['tWéenty', 'tWéenty', 'tWéenty', 'tWéenty', 'tWéenty', 'tWéenty', 'tWéenty', 'tWéenty', 'tWéenty', 'tWéenty']

Most repeated CAPTCHA result: tWéenty
Move the slider to the lower of -twenty-four: or tWéenty.
🔢 Số cần kéo đến: 34
➡️ Đã kéo slider.
📩 Đã nhấn Submit.
⚠️ Thử lần 3: Captcha 

In [67]:
# Mở trình duyệt
print("🔓 Mở trình duyệt để nhập captcha...")
driver = webdriver.Chrome()
driver.get("https://trade.ec.europa.eu/access-to-markets/en/captcha?redirectTo=%2Fen%2Fsearch%3Fproduct%3D0101%26origin%3DVN%26destination%3DAT")
print("⌨️ Vui lòng nhập captcha, sau đó đóng trình duyệt nếu không cần chạy lại.")

🔓 Mở trình duyệt để nhập captcha...
⌨️ Vui lòng nhập captcha, sau đó đóng trình duyệt nếu không cần chạy lại.


In [68]:
slider = driver.find_element(By.CLASS_NAME, "wt-noconflict")
slider_width = slider.size["width"]

In [ ]:
# Find the input element
input_element = driver.find_element(By.CLASS_NAME, "wt-noconflict")
# Use JavaScript to add or change the "value" attribute
driver.execute_script("arguments[0].setAttribute('value', '42');", input_element)

In [74]:
# Trigger "input" and "change" events to update the UI
driver.execute_script("arguments[0].dispatchEvent(new Event('input'));", input_element)
driver.execute_script("arguments[0].dispatchEvent(new Event('change'));", input_element)

In [ ]:
submit_button = driver.find_element(By.CSS_SELECTOR, 'button.ecl-button--call[type="submit"]')
driver.execute_script("arguments[0].click();", submit_button)
